# Classifier training

In [43]:
import pandas as pd
import numpy as np
import tensorflow as tf
import scipy.stats as sm
import sklearn

In [34]:
names=['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']

df = pd.read_csv('heart.csv')

In [35]:
y = df.iloc[:, -1].values
X = df.iloc[:, :-1].values

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print("X_train:", X_train.shape, "X_test:", X_test.shape)
print("y_train:", y_train.shape, "y_test:", y_test.shape)

X_train: (242, 13) X_test: (61, 13)
y_train: (242,) y_test: (61,)


In [37]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [54]:
# Importing the Keras libraries and packages
import tensorflow as tf
from tensorflow import keras

classifier = keras.Sequential()

# Adding the input layer and the first hidden layer
classifier.add(keras.layers.Dense(6, bias_initializer = 'uniform', activation = 'relu', input_dim = 13))

# Adding the second hidden layer
classifier.add(keras.layers.Dense(13, activation = 'relu'))

# Adding the output layer
classifier.add(keras.layers.Dense(1, activation = 'sigmoid'))

In [55]:
# Compiling Neural Network
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting our model 
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
242/242 [==============================] - 0s 2ms/step - loss: 0.7795 - acc: 0.5372
Epoch 2/100
242/242 [==============================] - 0s 164us/step - loss: 0.6766 - acc: 0.5992
Epoch 3/100
242/242 [==============================] - 0s 198us/step - loss: 0.6127 - acc: 0.6653
Epoch 4/100
242/242 [==============================] - 0s 209us/step - loss: 0.5652 - acc: 0.7231
Epoch 5/100
242/242 [==============================] - 0s 204us/step - loss: 0.5274 - acc: 0.7645
Epoch 6/100
242/242 [==============================] - 0s 209us/step - loss: 0.4990 - acc: 0.7769
Epoch 7/100
242/242 [==============================] - 0s 265us/step - loss: 0.4744 - acc: 0.7934
Epoch 8/100
242/242 [==============================] - 0s 218us/step - loss: 0.4558 - acc: 0.7934
Epoch 9/100
242/242 [==============================] - 0s 190us/step - loss: 0.4403 - acc: 0.8058
Epoch 10/100
242/242 [==============================] - 0s 199us/step - loss: 0.4280 - acc: 0.8058
Epoch 11/100
242/242 

In [56]:
y_pred = classifier.predict(X_test)
y_pred = y_pred.flatten().round()
print(y_test)
print(y_pred)

[1 1 0 0 1 1 1 0 0 1 1 0 1 1 0 1 1 0 0 0 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 0 1 0 0 0 1 0 0 0 0 0 1 1 1 1 1 0 0 1 0]
[1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0.
 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0.]


In [57]:
y_actu = pd.Series(y_test, name='Actual')
y_pred = pd.Series(y_pred, name='Predicted')

df_confusion = pd.crosstab(y_actu, y_pred)
print(df_confusion)

Predicted  0.0  1.0
Actual             
0           22    2
1            6   31


In [58]:
accuracy = 100 * np.sum(y_test == y_pred) / len(y_test)
print("Test accuracy:", accuracy)

y_train_pred = classifier.predict(X_train).flatten().round()
accuracy_train = 100 * np.sum(y_train == y_train_pred) / len(y_train)
print("Train accuracy:", accuracy_train)

Test accuracy: 86.88524590163935
Train accuracy: 91.32231404958678


In [47]:
def predict(classifier):
    # Train the model using the training sets
    classifier.fit(X_train, y_train)  

    # Predict the response for test dataset
    return classifier.predict(X_test)  
    
def get_accuracy():
    chi = sm.chi2_contingency(pd.crosstab(index=y_test, columns=y_pred))
    effect = np.sqrt(chi[0]/(len(y_test)*1))

    print("Chi-squared effect:", effect)
    print("Sklearn accuracy:", sklearn.metrics.accuracy_score(y_test, y_pred))
    print("\nConfusion matrix (predicted vs actual)")
    print(pd.DataFrame(np.c_[sklearn.metrics.confusion_matrix(y_test, y_pred)]))  

In [51]:
from sklearn.tree import DecisionTreeClassifier  

classifier = DecisionTreeClassifier()  

# Train the model and predict against the validation set
y_pred = predict(classifier)  

# Get accuracy results
get_accuracy()

Chi-squared effect: 0.27871621621621623
Sklearn accuracy: 0.6721311475409836

Confusion matrix (predicted vs actual)
    0   1
0  14  10
1  10  27


In [52]:
from sklearn.svm import SVC

# Create an svm Classifier
classifier = SVC(kernel='linear') # Linear Kernel

# Train the model and predict against the validation set
y_pred = predict(classifier)  

# Get accuracy results
get_accuracy()

Chi-squared effect: 0.6908783783783784
Sklearn accuracy: 0.8688524590163934

Confusion matrix (predicted vs actual)
    0   1
0  20   4
1   4  33
